In [ ]:
#@title Mount google drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

%cd './drive/MyDrive/gaze_estimation'

In [ ]:
import time
import copy
import torch
import random
import torchvision
import numpy as np

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from tqdm.auto import tqdm

from util import make_reproducibility, TensorDataset, convert_to_xyz, mae
from gazenet.gazenet import gazenet
from networks import *

In [ ]:
#@title Load preprocessed & subsampled data (LOOCV)

ids = np.load('../ut_dataset/10-fold/within_ids.npy', allow_pickle=True)
images = torch.as_tensor(np.load('../ut_dataset/10-fold/within_images.npy'), dtype=torch.float)
hps = torch.as_tensor(np.load('../ut_dataset/10-fold/within_hps.npy'), dtype=torch.float)
gazes = torch.as_tensor(np.load('../ut_dataset/10-fold/within_gazes.npy'), dtype=torch.float)

In [ ]:
device = torch.device('cuda:0')
seed = 10

experiment_name = 'ut_10fold'
res_list = []

In [ ]:
for fold in range(10) :

    train_ids = np.concatenate([ids[:fold], ids[(fold+1):]]).reshape(-1)
    train_images = torch.cat([images[:fold], images[(fold+1):]]).reshape(-1,1, 36,60)
    train_hps = torch.cat([hps[:fold], hps[(fold+1):]]).reshape(-1,2)
    train_gazes = torch.cat([gazes[:fold], gazes[(fold+1):]]).reshape(-1,2)

    test_ids = ids[fold]
    test_images = images[fold].unsqueeze(1)
    test_hps = hps[fold]
    test_gazes = gazes[fold]

    res_list.append(gazenet(
        train_ids, train_images, train_hps, train_gazes,
        test_ids, test_images, test_hps, test_gazes,
        VGG16_rev.VGG16, max_iter = 15000,
        device=device, experiment_name = f'{experiment_name}_{fold}', SEED =100 + fold, verbose=False, large_test=True))

